In [ ]:
!pip install -e ../.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# This to make sure that the changes in the module are reloaded
%reload_ext autoreload

In [ ]:
# Platform configs
url = "https://api.jeroennelen.nl"
project_id = "66277759e1eacb39208e10ed"

project_setup = {
    "platform_url": url,
    "project_id": project_id
}

In [ ]:
# checkpoint = "mlabonne/Gemmalpaca-2B"
# "meta-llama/Meta-Llama-3-8B"
device = "cpu"
checkpoint = "bigscience/bloom-1b7"

model_setup = {
    "checkpoint": checkpoint,
    "model_type": "dec",
    "embeddings": "transformer.word_embeddings"
}

run_config = {
    "device": device,
    "max_new_tokens": 100,
    # "temperature": 1, # Not
    # e that if you add this, then you need to add "do_sample": True
    # "do_sample": True
}

In [ ]:
import helmet

In [ ]:
model = helmet.from_pretrained(project_setup=project_setup, model_setup=model_setup, run_config=run_config)

In [ ]:
# !pip install huggingface_hub ipywidgets -q

In [ ]:
# This might be needed to load the model
# from huggingface_hub import login
# login()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Helsinki-NLP/opus-100", name="en-nl", split='test[:10]')

In [ ]:
dataset[1]

In [ ]:
# prompt = dataset["validation"][1]
# # print(prompt)
# # prompt = prompt["question"]

In [ ]:
source_lang = "en"
target_lang = "nl"
prefix = "Translate English to Dutch: "

def prompt_builder(sample):
    return prefix + sample['translation'][source_lang]


prompt = prompt_builder(dataset[1])
prompt

In [ ]:
predict_config = {
    "max_new_tokens": 10,
    "generate_explanations": False,
}

# prompt = "Translate English to Italian: I am a student."
result = model.predict(prompt, **predict_config)
result

In [ ]:
model.saliency_explainer("6627781fe1eacb39208e10ee")